In [60]:
from langchain_community.llms import Ollama
from openai import OpenAI
import os
from IPython.display import display, HTML, clear_output


def p(str, color):
    display(
        (
            HTML(
                f"""
      <div style="margin: 10px; padding: 10px; border: 1px solid black; background-color:#fff5e3; ">
      <span style="font-size: 14px; font-weight: 500; color: {color};">{str}</span>
      </div>
    """
            )
        )
    )


class OpenAi:
    def __init__(self):
        self.client = OpenAI(
            api_key=os.environ.get("OPENAI_API_KEY"),
        )

    def invoke(self, system, user):
        completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": system,
                },
                {
                    "role": "user",
                    "content": user,
                },
            ],
            model="gpt-3.5-turbo",
        )
        return completion.choices[0].message.content


llm = Ollama(model="gemma:2b")
result = llm.invoke("test")
p(result, "green")
open_ai = OpenAi()

In [61]:
import pprint

pp = pprint.pprint

setting = "a family drama set in an english manor"


def generate_name(past_names):
    return llm.invoke(
        f"Generate a plausible but.  Dont use any of the following: {",".join(past_names)}. ONLY RETURN THE NAME"
    )


def generate_characters(num_characters, setting):
    character_gen_prompt = f"""
    Generate a character in a play. The setting is {setting}
    The character should have:
    - a name, age and gender
    - 2 sentences detailing their background
    - 3 sentences detailing how they interact with others
    - 1 sentence describing an event that was deeply formative
    - 1 sentence explicitly detailing a secret they dont want to reveal.  Explain what the secret is
    """

    characters = []
    for i in range(num_characters):
        char = llm.invoke(character_gen_prompt)
        characters.append(char)
    return characters


characters = generate_characters(3, setting)

p("\n".join(characters), "purple")

In [62]:
def generate_conversation(characters, setting, summary, event):
    recent_event = event if event else "No recent events"
    conversation_prompt = f"""
    You are helping to create a play about {setting}
    
    Generate a conversation between the following characters {"\n\n".join(characters)}.

    The conversation should be dramatic, and based on the history of the characters, their motivations, and their secrets.

    A summary of the events and conversation so far: {summary}.

    A recent event: ${recent_event}

    ONLY generate a few sentences of conversation between one or more of the parties.
  """

    conversation = llm.invoke(conversation_prompt)
    p(f"\n{conversation}\n", "magenta")
    return conversation


def generate_dramatic_event(characters, setting, summary):
    event_prompt = f"""
    You are helping to create a play about {setting}

    Characters in the play: {"\n\n".join(characters)}
    
    Generate a dramatic event that changes the course of the plot.
  """
    event = llm.invoke(event_prompt)
    p(f"\nEvent: ${event}", "black")
    return event


def summarize(dialog):
    return llm.invoke(
        f"""
      Summarize the following {"\n".join(dialog)}
    """
    )


def generate_scene(interactions, characters, setting):
    event = None
    summary = ""
    scene = []
    for i in range(interactions):
        p(f"Interaction: {i}", "green")
        conversation = generate_conversation(characters, setting, summary, event)
        summary = summarize(scene)
        event = generate_dramatic_event(characters, setting, summary)
        scene.append(conversation)
        scene.append(event)

    return scene


scene = generate_scene(5, characters, setting)

In [69]:
with open("scene.txt", "w") as f:
    for line in scene:
        f.write(line + "\n")

scene_prompt = f"""
  

  Take the following scene that includes conversation and events, and return them in screenplay format:

  {"\n".join(scene)}
"""

screenplay = open_ai.invoke(
    "You are helping to create a play about {setting}.", scene_prompt
)

with open("screenplay.txt", "w") as f:
    f.write(screenplay + "\n")